In [3]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
from torchsummary import summary

In [2]:
#activation fumction ReLU and its derivative
def ReLu_d(x):
    return np.where(x < 0, 0, 1)
def ReLu(x):
    return np.where(x < 0, 0, x)

In [4]:
#Tensor->（multidimensional array）

#d input nodes k hidden layer
K = 10
d = 10
batch_size = 1

X = Variable(torch.rand(batch_size,d),requires_grad=True)
y = torch.tensor([[torch.sum(X**2)/d]])

In [5]:
class NN1(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        global z,z_relu
        z = []
        z_relu = []
        super(NN1, self).__init__()
        self.layer = nn.Sequential()
        self.layer.add_module("h_1", nn.Linear(input_size, hidden_size))
        
        z.append(self.layer(X))
        z_relu.append(self.layer(X))
        
        for i in range(K-1):
             index = str(i+2)
             self.layer.add_module("h_"+index, nn.Linear(hidden_size, hidden_size))
             z.append(self.layer(X)) 
             self.layer.add_module("h_"+index+"_relu"+index,nn.ReLU())
             z_relu.append(self.layer(X))
                
        self.layer.add_module("h_last", nn.Linear(hidden_size, num_classes))
        z.append(self.layer(X))
        #self.layer.add_module("h_1ast_relu"+index,nn.ReLU())
        z_relu.append(self.layer(X))

    def forward(self, X):
        y = self.layer(X)
        return y
        

In [6]:
model = NN1(d,2*d+1,1)

In [7]:
y_pred = model(X)
loss = (y_pred - y)**2
loss.item()


0.08377084136009216

In [8]:
y_pred

tensor([[0.0924]], grad_fn=<AddmmBackward>)

In [9]:
summary(model,(1,10))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 21]             231
            Linear-2                [-1, 1, 21]             462
              ReLU-3                [-1, 1, 21]               0
            Linear-4                [-1, 1, 21]             462
              ReLU-5                [-1, 1, 21]               0
            Linear-6                [-1, 1, 21]             462
              ReLU-7                [-1, 1, 21]               0
            Linear-8                [-1, 1, 21]             462
              ReLU-9                [-1, 1, 21]               0
           Linear-10                [-1, 1, 21]             462
             ReLU-11                [-1, 1, 21]               0
           Linear-12                [-1, 1, 21]             462
             ReLU-13                [-1, 1, 21]               0
           Linear-14                [-1

In [10]:
loss.backward()

--------------------

In [1]:
#for param in model.named_parameters():
    #print(param)

In [20]:
#save auto w,b
auto_w =[]
auto_b =[]
i = 0

for param in model.parameters():
    if (i%2==0):
        #auto_w.append(param.grad)
        auto_w.append(np.round(np.array(param.grad).astype(np.double),5))
    if (i%2==1):
        #auto_b.append(param.grad)
         auto_b.append(np.round(np.array(param.grad).astype(np.double),5))
    i+=1

-----------------------

In [21]:
#手动grad share the same weight and bias with auto_forward
parameters = list(model.parameters())

In [22]:
#manual forward
def feedforward(X,parameters):
    global z_manual,z_relu_manual
    z_manual=[]
    z_relu_manual=[]
    z_manual.append(torch.from_numpy(np.dot(X.detach().numpy(),(parameters[0].detach().numpy().T))+parameters[1].detach().numpy()))
    z_relu_manual.append(z_manual[0])
    
    for i in range(0,10):
        z_manual.append(torch.from_numpy(np.dot(z_relu_manual[i],(parameters[(i+1)*2].detach().numpy().T))+parameters[(i+1)*2+1].detach().numpy()))
        z_relu_manual.append(ReLu(z_manual[i+1]))
    z_relu_manual[-1]=z_manual[-1]
    return z_relu_manual[-1]
y_pred = feedforward(X,parameters)

In [33]:
diff_z=0
for i in range(len(z)):
    diff_z = diff_z+ np.absolute(sum(sum(z[i].detach().numpy()-z_manual[i].detach().numpy())))
diff_z

0.0

In [47]:
diff_z_relu=0
for i in range(len(z_relu)):
    diff_z_relu = diff_z_relu+ np.absolute(sum(sum(z_relu[i].detach().numpy()-np.array(z_relu_manual[i]))))
diff_z_relu

0.0

In [23]:
#求对z的的倒数
# dL/dv5
d_z = []
d_w = []
torch.set_printoptions(precision=5)
grad_y_pred = 2.0 * (y_pred - y) #[1,1]
grad_z_y = grad_y_pred.detach().numpy()#*ReLu_d(y_pred)#dL/dz5,5)
grad_w_y = grad_z_y*z_relu_manual[-2]#.detach().numpy()
d_z.append(grad_z_y)
d_w.append(grad_w_y)
#from the last hidden layer
for i in range (9):#i=1,2,3,4,5.... p:-4,-6,-8
    grad_z_i = np.dot(d_z[i],ReLu_d(z_manual[-i-2])*parameters[-(i+1)*2].detach().numpy())
    d_z.append(grad_z_i)
    grad_w_i = np.dot(grad_z_i.T,z_relu_manual[-3-i])#.detach().numpy() )#-4,-5,-6
    d_w.append(grad_w_i)
    
#first layer
grad_z_1 = np.dot(d_z[9],ReLu_d(z_manual[-11])*parameters[-20].detach().numpy())
d_z.append(grad_z_1)
grad_w_1 = np.dot(grad_z_1.T,X.detach().numpy())
d_w.append(grad_w_1)


-------------

In [24]:
for i in range(len(d_w)):
    d_w[i]= np.round(np.array(d_w[i]).astype(np.double),5)
    d_z[i]= np.round(np.array(d_z[i]).astype(np.double),5)

In [25]:
d_w.reverse()

In [26]:
d_z.reverse()

In [222]:
#chaeck value difference between autograd and mygrad

In [27]:
diff_w=0
for i in range(len(d_w)):
    diff_w = diff_w+ np.absolute(sum(sum(d_w[i]-auto_w[i])))

In [28]:
diff_w

2.0000000000000012e-05

In [29]:
diff_b=0
for i in range(len(d_z)):
    diff_b = diff_b+ np.absolute(sum(sum(d_z[i]-auto_b[i])))
diff_b

3.3881317890172014e-21

In [214]:
with open('torch_autograd.dat', 'w') as f:
    f.write('autograd_w' + '\n')
    i = 1
    for w in auto_w:
        f.write('weight_' +str(i)+ '\n')
        f.write(str(w)+'\n')
        i+=1
    f.write('autograd_b' + '\n')
    i = 1
    for b in auto_b:
        f.write('bias_' +str(i)+ '\n')
        f.write(str(b)+'\r\n')
        i+=1
    

In [215]:
with open('my_autograd.dat', 'w') as f:
    f.write('my_w' + '\n')
    i = 1
    for w in d_w:
        f.write('weight_' +str(i)+ '\n')
        f.write(str(w)+'\n')
        i+=1
    f.write('my_b' + '\n')
    i = 1
    for b in d_z:
        f.write('bias_' +str(i)+ '\n')
        f.write(str(b)+'\r\n')
        i+=1
    